To be able to use my already existing python environment, I had to give Visual Studio Code the path to my environments folder. 

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f"Length of dataset: {len(text)} characters.")

# There are a total of 65 unique characters in the dataset.
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print("".join(chars))

# We will tokenize our vocabulary by building a character level language model. We will represent each
# character as an integer. Sub-word tokenizers are also possible (chat-gpt uses tiktoken)
# We first create a mapping from characters to integers using a dictionary
chtoi = {ch:i for i,ch in enumerate(chars)}
itoch = {i:ch for i,ch in enumerate(chars)}

def encode(s):  
    return [chtoi[ch] for ch in s] # Take a string, output list of integers.

def decode(list_int):
    return "".join([itoch[i] for i in list_int]) # Take a list of integers, output string.

Length of dataset: 1115394 characters.
65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
# We now encode entire "input.txt" and save it in a torch tensor.
import torch
import torch.nn as nn
from torch.nn import functional as F
data = torch.tensor(encode(text))

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

When we train a transformer, we only work with random chunks we take from the dataset. 

In a chunk of 9 characters, there are 8 training examples of increasing context length. Maximum context length we train with is given by block_size. This is useful for inference as the transformer is used to working with varying context lengths. For inference, we have to divide inputs larger than block_size into chunks. 

In [5]:
block_size = 8

print("CONTEXT")
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When {context} is the context, the target is {target}.")

CONTEXT
When tensor([18]) is the context, the target is 47.
When tensor([18, 47]) is the context, the target is 56.
When tensor([18, 47, 56]) is the context, the target is 57.
When tensor([18, 47, 56, 57]) is the context, the target is 58.
When tensor([18, 47, 56, 57, 58]) is the context, the target is 1.
When tensor([18, 47, 56, 57, 58,  1]) is the context, the target is 15.
When tensor([18, 47, 56, 57, 58,  1, 15]) is the context, the target is 47.
When tensor([18, 47, 56, 57, 58,  1, 15, 47]) is the context, the target is 58.


In [6]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for prediction?

def get_batch(split):
    """
    We obtain a context and target tensor of size (batch_size, block_size)
    """
    data = train_data if split=="train" else val_data
    ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,))

    # We now turn horizontally
    X = torch.vstack([data[i:i+block_size] for i in ix])
    Y = torch.vstack([data[i+1:i+block_size+1] for i in ix])

    return X,Y

# BIGRAM

Bigrams are a very simple model. They simply use a look-up table and no context. They use only the current character to predict the next. 

The objective of the generate() function is to extend the (batch_size, block_size) horizontally and predict more tokens. Gets (B,T) -> (B,T+1)

min 38

logit: output of a neuron without applying activation function.

In [7]:
idx = torch.tensor([[0,4,6,2],
                    [3,7,8,9]])  # size: (batch_size, block_size)

token_embedding_table = nn.Embedding(vocab_size, vocab_size)
logits = token_embedding_table(idx) # size: (batch_size, block_size, vocab_size)

print(logits.shape)

torch.Size([2, 4, 65])


In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # First input is vocab_size. Second input is the size of the encoded representation for each word. 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        """P
        Embeddings are used when working with categorical data. Often used to map discrete tokens (such as characters in a text)
        to continous vectors.

        Useful link.
        https://spltech.co.uk/in-pytorch-what-is-nn-embedding-for-and-how-is-it-different-from-one-hot-encding-for-representing-categorical-data/?utm_content=cmp-true
        """
        # idx and targets are tensors of size (batch_size, block_size)
        logits = self.token_embedding_table(idx)   # size: (batch_size, block_size, vocab_size)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        We call this function to generate new characters.
        """
        for _ in range(max_new_tokens):
            # We first get the predictions
            logits, loss = self(idx)  # (B,T,C)
            
            # Here, we are interested in using all the given context.
            logits = logits[:, -1, :]  # (B,C)

            # We then apply softmax to get probabilities.
            probs = F.softmax(logits, dim=-1)  # (B,C)

            # We now sample from the probabilities
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)

            # Finally, we append
            idx = torch.hstack([idx, idx_next])  # (B, T+1)

        return idx

@torch.no_grad()
def estimate_loss(eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 1000
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss(eval_iters)
        print(f"iter: {iter}  train_loss: {losses['train']:.4f}  val_loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

iter: 0  train_loss: 4.6162  val_loss: 4.6193
iter: 1000  train_loss: 2.4922  val_loss: 2.5184
iter: 2000  train_loss: 2.4764  val_loss: 2.4926
iter: 3000  train_loss: 2.4600  val_loss: 2.4883
iter: 4000  train_loss: 2.4560  val_loss: 2.4900


In [10]:
context = "Thou art"
context = torch.tensor(encode(context)).unsqueeze(0)
print(context.shape)

text = decode(m.generate(context, 200)[0].tolist())
print(text)

torch.Size([1, 8])
Thou art, t, ine t nghanstl stomy t t ncthever hastind e fr balawin fas, mirengered fap ig' had yocrs.
IIncearuld st n,

RDort saspe ariasth o towsixf my hin angal der lled
S m
LUKEThyour s.

ARCosarnevik e blode d bllo ttag hir wat? t t s
BEO: s o, ad.

Cl IIORDomart k'd CENCEMIUS:
OMu tr'e;
A: the thadswoJULEO scine to: y the. anges, War pETrmevanemy gacouthe st wis.
THENSThe; cksimer r ar se, sh HAnt e


In [13]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

b = 1
t = 2
print(x[b, :t+1, :].shape)

torch.Size([3, 2])


### Simple self-attention

What we want to do now is to code up the most simple type of attention. Where for each batch independently, for each target t, we take the mean of the previous context. We will refer to this tensor as xbow (bag of words). The name comes from the fact that averaging is essentially just throwing all the words into a bag.

In [55]:
# x[b,t] = mean_{i <= t} x[b,i]
xbow = torch.zeros_like(x)

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1, :] # (t, C)
        xbow[b,t] = torch.mean(xprev, dim=0)
print(xbow[0])

# There is a way to make this code much more efficient. We can perform this weighted aggregation with matrix multiplication. 
low_triangular_ones = torch.tril(torch.ones((T,T)))
divisor = torch.arange(1, T+1, device=device).unsqueeze(1)
low_triangular_ones /= divisor
print(low_triangular_ones)

# We perform matrix multiplication over each batch independently. We can use einsum for tensor operations.
xbow_einsum = torch.einsum('ij, ajk-> aik', low_triangular_ones, x)

# For matmul we have to introduce intermediate tensors. 
low_triangular_ones_batch = low_triangular_ones.unsqueeze(0).repeat(B, 1, 1)
xbow_matmul = torch.matmul(low_triangular_ones_batch , x)

print(torch.allclose(xbow, xbow_einsum))
print(torch.allclose(xbow, xbow_matmul))

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
True
False
